In [1]:
# Implementation of a simple MLP network with one hidden layer. Tested on the iris data set.
# Requires: numpy, sklearn>=0.18.1, tensorflow>=1.0

# NOTE: In order to make the code simple, we rewrite x * W_1 + b_1 = x' * W_1'
# where x' = [x | 1] and W_1' is the matrix W_1 appended with a new row with elements b_1's.
# Similarly, for h * W_2 + b_2

In [57]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import datetime
import math
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [6]:
# loading in the data

train_features = pd.read_csv('dengue_features_train.csv')
train_labels = pd.read_csv('dengue_labels_train.csv')
test_features = pd.read_csv('dengue_features_test.csv')

# splitting by city

train_features_sj = train_features[:936]
train_features_iq = train_features[936:]

train_labels_sj = train_labels[:936]
train_labels_iq = train_labels[936:]

test_features_sj = test_features[:260]
test_features_iq = test_features[260:]

# introducing a datetime feature that contains the unix timestamp

def get_timestamp(features):
    dt = []
    for date in features['week_start_date']:
        dt.append(time.mktime(datetime.datetime.strptime(date, '%Y-%m-%d').timetuple()) / 1000)
    return dt

sj_times = get_timestamp(train_features_sj)
iq_times = get_timestamp(train_features_iq)
sj_times_test = get_timestamp(test_features_sj)
iq_times_test = get_timestamp(test_features_iq)

train_features_sj['datetime'] = sj_times
train_features_iq['datetime'] = iq_times
test_features_sj['datetime'] = sj_times_test
test_features_iq['datetime'] = iq_times_test

# dropping the city feature and the other features previously used to describe time, now that I have a timestamp

train_features_sj = train_features_sj.drop(train_features_sj.columns[[0,1,2,3]], axis=1)
train_features_iq = train_features_iq.drop(train_features_iq.columns[[0,1,2,3]], axis=1)

test_features_sj = test_features_sj.drop(test_features_sj.columns[[0,1,2,3]], axis=1)
test_features_iq = test_features_iq.drop(test_features_iq.columns[[0,1,2,3]], axis=1)

# filling in missing data

train_features_sj.fillna(method='bfill', inplace=True)
train_features_iq.fillna(method='bfill', inplace=True)

test_features_sj.fillna(method='bfill', inplace=True)
test_features_iq.fillna(method='bfill', inplace=True)

train_labels_sj = train_labels_sj.total_cases
train_labels_iq = train_labels_iq.total_cases

C:\Users\8050116\AppData\Local\conda\conda\envs\iX2017\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\8050116\AppData\Local\conda\conda\envs\iX2017\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\8050116\AppData\Local\conda\conda\envs\iX2017\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [77]:
# selecting a subset of the features
X_sj = train_features_sj[['precipitation_amt_mm', 'reanalysis_air_temp_k', 'reanalysis_avg_temp_k', 
                          'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 
                          'reanalysis_precip_amt_kg_per_m2', 'reanalysis_relative_humidity_percent', 
                          'reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 
                          'reanalysis_tdtr_k', 'station_precip_mm', 'datetime']]
y_sj = train_labels_sj
X_iq = train_features_iq[['precipitation_amt_mm', 'reanalysis_air_temp_k', 'reanalysis_avg_temp_k', 
                          'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 
                          'reanalysis_precip_amt_kg_per_m2', 'reanalysis_relative_humidity_percent', 
                          'reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 
                          'reanalysis_tdtr_k', 'station_precip_mm', 'datetime']]
y_iq = train_labels_iq

# test X values
X_sj_test = test_features_sj[['precipitation_amt_mm', 'reanalysis_air_temp_k', 'reanalysis_avg_temp_k', 
                              'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 
                              'reanalysis_precip_amt_kg_per_m2', 'reanalysis_relative_humidity_percent', 
                              'reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 
                              'reanalysis_tdtr_k', 'station_precip_mm', 'datetime']]

X_iq_test = test_features_iq[['precipitation_amt_mm', 'reanalysis_air_temp_k', 'reanalysis_avg_temp_k', 
                              'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 
                              'reanalysis_precip_amt_kg_per_m2', 'reanalysis_relative_humidity_percent', 
                              'reanalysis_sat_precip_amt_mm', 'reanalysis_specific_humidity_g_per_kg', 
                              'reanalysis_tdtr_k', 'station_precip_mm', 'datetime']]

In [78]:
# new feature, 0 if cold (<300 Kelvin), 1 if warm

def is_warm(features):
    warm = []
    for observation in features['reanalysis_avg_temp_k']:
        if observation < 300:
            warm.append(0)
        else:
            warm.append(1)
    return warm

warmth_sj = is_warm(X_sj)
warmth_iq = is_warm(X_iq)
warmth_sj_test = is_warm(X_sj_test)
warmth_iq_test = is_warm(X_iq_test)

X_sj.warmth = warmth_sj
X_iq.warmth = warmth_iq
X_sj_test.warmth = warmth_sj_test
X_iq_test.warmth = warmth_iq_test

In [79]:
# splitting data into training set and validation set

X_sj_train, X_sj_val, y_sj_train, y_sj_val = train_test_split(X_sj, y_sj, test_size=0.2, random_state=41)

X_iq_train, X_iq_val, y_iq_train, y_iq_val = train_test_split(X_iq, y_iq, test_size=0.2, random_state=41)


In [80]:
RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

In [81]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

In [105]:
def forwardprop(X, w_1, w_2, w_3):
    h = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    h2 = tf.nn.sigmoid(tf.matmul(h, w_2))
    yhat = tf.matmul(h, w_3)  # The \varphi function
    return yhat

In [83]:
# Layer's sizes
x_size = X_sj_train.shape[1]   # Number of input nodes: 13 features
h_size = 256                # Number of hidden nodes
y_size = 1

In [84]:
# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

In [106]:
# Weight initializations
w_1 = init_weights((x_size, h_size))
w_2 = init_weights((h_size, h_size))
w_3 = init_weights((h_size, y_size))

In [107]:
# Forward propagation
yhat    = forwardprop(X, w_1, w_2, w_3)
predict = tf.to_int64(yhat)

In [108]:
# Backward propagation
cost    = tf.losses.mean_squared_error(labels=y, predictions=yhat)
updates = tf.train.GradientDescentOptimizer(0.0000003).minimize(cost)

In [88]:
# transposing the label vectors
y_sj_train = y_sj_train.as_matrix()
y_iq_train = y_iq_train.as_matrix()

y_sj_train = y_sj_train[:, None]
y_iq_train = y_iq_train[:, None]

In [89]:
y_sj = y_sj.as_matrix()
y_iq = y_iq.as_matrix()

y_sj = y_sj[:, None]
y_iq = y_iq[:, None]

In [109]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
epochs = 100
for epoch in range(epochs):
        # Train with each example
        for i in range(len(X_sj_train)):
            sess.run(updates, feed_dict={X: X_sj_train[i: i + 1], y: y_sj_train[i: i + 1]})

        train_accuracy = metrics.mean_absolute_error(y_sj_train,
                                                     sess.run(predict, feed_dict={X: X_sj_train}))
        test_accuracy  = metrics.mean_absolute_error(y_sj_val,
                                                     sess.run(predict, feed_dict={X: X_sj_val}))

        if epoch % 10 == 0 or epoch == epochs-1:
            print("Epoch = %d, train loss = %.2f, test loss = %.2f"
                  % (epoch + 1, train_accuracy, test_accuracy))

Epoch = 1, train loss = 33.21, test loss = 28.28
Epoch = 11, train loss = 25.60, test loss = 21.24
Epoch = 21, train loss = 26.32, test loss = 22.59
Epoch = 31, train loss = 27.40, test loss = 23.91
Epoch = 41, train loss = 28.11, test loss = 24.78
Epoch = 51, train loss = 28.50, test loss = 25.24
Epoch = 61, train loss = 28.91, test loss = 25.72
Epoch = 71, train loss = 28.91, test loss = 25.72
Epoch = 81, train loss = 29.35, test loss = 26.22
Epoch = 91, train loss = 29.35, test loss = 26.22
Epoch = 100, train loss = 29.35, test loss = 26.22


In [110]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
epochs = 100
for epoch in range(epochs):
        # Train with each example
        for i in range(len(X_iq_train)):
            sess.run(updates, feed_dict={X: X_iq_train[i: i + 1], y: y_iq_train[i: i + 1]})

        train_accuracy = metrics.mean_absolute_error(y_iq_train,
                                                     sess.run(predict, feed_dict={X: X_iq_train}))
        test_accuracy  = metrics.mean_absolute_error(y_iq_val,
                                                     sess.run(predict, feed_dict={X: X_iq_val}))
        
        if epoch % 10 == 0 or epoch == epochs-1:
            print("Epoch = %d, train loss = %.2f, test loss = %.2f"
                  % (epoch + 1, train_accuracy, test_accuracy))

Epoch = 1, train loss = 7.33, test loss = 8.49
Epoch = 11, train loss = 6.25, test loss = 7.41
Epoch = 21, train loss = 5.82, test loss = 7.09
Epoch = 31, train loss = 5.80, test loss = 7.05
Epoch = 41, train loss = 5.80, test loss = 7.05
Epoch = 51, train loss = 5.96, test loss = 7.12
Epoch = 61, train loss = 5.96, test loss = 7.12
Epoch = 71, train loss = 5.96, test loss = 7.12
Epoch = 81, train loss = 5.96, test loss = 7.12
Epoch = 91, train loss = 6.27, test loss = 7.30
Epoch = 100, train loss = 6.27, test loss = 7.30


#### Retraining on all data for submission

In [111]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
epochs = 100
for epoch in range(epochs):
        # Train with each example
        for i in range(len(X_sj)):
            sess.run(updates, feed_dict={X: X_sj[i: i + 1], y: y_sj[i: i + 1]})

        train_accuracy = metrics.mean_absolute_error(y_sj,
                                                     sess.run(predict, feed_dict={X: X_sj}))
        
        if epoch % 10 == 0 or epoch == epochs-1:
            print("Epoch = %d, train loss = %.2f"
                  % (epoch + 1, train_accuracy))
    
final_predict_sj = sess.run(predict, feed_dict={X: X_sj_test})

Epoch = 1, train loss = 32.22
Epoch = 11, train loss = 24.73
Epoch = 21, train loss = 25.82
Epoch = 31, train loss = 27.06
Epoch = 41, train loss = 27.45
Epoch = 51, train loss = 27.84
Epoch = 61, train loss = 27.84
Epoch = 71, train loss = 27.84
Epoch = 81, train loss = 27.84
Epoch = 91, train loss = 27.84
Epoch = 100, train loss = 27.84


In [112]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
epochs = 100
for epoch in range(epochs):
        # Train with each example
        for i in range(len(X_iq)):
            sess.run(updates, feed_dict={X: X_iq[i: i + 1], y: y_iq[i: i + 1]})

        train_accuracy = metrics.mean_absolute_error(y_iq,
                                                     sess.run(predict, feed_dict={X: X_iq}))
        
        if epoch % 10 == 0 or epoch == epochs-1:
            print("Epoch = %d, train loss = %.2f"
                  % (epoch + 1, train_accuracy))
    
final_predict_iq = sess.run(predict, feed_dict={X: X_iq_test})

Epoch = 1, train loss = 7.57
Epoch = 11, train loss = 6.21
Epoch = 21, train loss = 6.08
Epoch = 31, train loss = 6.05
Epoch = 41, train loss = 6.19
Epoch = 51, train loss = 6.19
Epoch = 61, train loss = 6.47
Epoch = 71, train loss = 6.47
Epoch = 81, train loss = 6.47
Epoch = 91, train loss = 6.47
Epoch = 100, train loss = 6.47


In [115]:
#Creating file with test cases to be tested
submission = pd.read_csv("submission_format.csv",
                         index_col=[0, 1, 2])

submission.total_cases = np.concatenate([final_predict_sj, final_predict_iq])
submission.to_csv("submission_MLP.csv")

In [116]:
sess.close()